In [2]:
import sqlite3 
mshp_db_path = r"C:\Users\lgarzia\Documents\GitHub\explainability\local_data\data\mshp.sqlite" 
conn = sqlite3.connect(mshp_db_path)

In [37]:
c = conn.cursor()
c.execute("DROP TABLE pre_feature_eng_crs_ii")

In [38]:
sql = '''	
    CREATE TABLE pre_feature_eng_crs_ii AS
    select DISTINCT dcr.acc_rpt_num, dcr.acc_uuid, 
	coalesce(dcr.Name, ii.Name) name,  
	coalesce(dcr."Personal Injury", ii."Injury Type") injury_type, 
	coalesce(dcr.Age, ii.Age) age,
	coalesce(dcr."Person City/State", ii."City/State") city_state,
	dcr."Safety Device" AS safety_device, 
    dcr.Date AS date, 
    dcr.Time AS time, 
    dcr.Troop AS troop, 
    ii.Gender AS gender,
	ii."Veh. #" as veh_num, 
	ii."Involvement" AS involvement
	from daily_crash_report dcr
	left outer join injury_information ii on dcr.acc_uuid = ii.acc_uuid  --13751, inner(10129)
	and dcr.Name = ii.Name 
	and dcr.Age = ii.Age 
	and dcr."Person City/State" = ii."City/State"
'''
# 	--and dcr."Personal Injury" = ii."Injury Type"

In [39]:
c = conn.cursor()
c.execute(sql)

In [40]:
import dataprep.connector as dbc
import urllib
table_name = "injury_information"
mshp_db_path = r"C:\Users\lgarzia\Documents\GitHub\explainability\local_data\data\mshp.sqlite" 
mshp_db_path = urllib.parse.quote(mshp_db_path)
mshp_db_conn = "sqlite://"+mshp_db_path

In [41]:
df = dbc.read_sql(mshp_db_conn, f"SELECT * FROM pre_feature_eng_crs_ii")

In [43]:
df.shape # (13720, 13), (13746, 13) <-- see more duplicates probably because of 'Unknowns & Juvenilles'

(13746, 13)

In [44]:
df.count()

acc_rpt_num      13746
acc_uuid         13746
name             13746
injury_type      13746
age              13746
city_state       13716
safety_device    13746
date             13746
time             13746
troop            13746
gender           10176
veh_num          10176
involvement      10176
dtype: int64

In [ ]:
# question is it possible to back into missing gender
# idea - get most from vehicle
3622

In [45]:
c = conn.cursor()
c.execute("DROP TABLE pre_feature_eng")

In [46]:
sql = ''' 
CREATE TABLE pre_feature_eng AS

select distinct cii.acc_rpt_num, cii.acc_uuid, cii.name, cii.injury_type, cii.age, 
cii.city_state, cii.safety_device, cii.date, cii.time, cii.troop, 
coalesce(cii.gender, vi."Driver Gender") AS gender, 
coalesce(cii.veh_num, vi."Veh. #") AS veh_num,
coalesce(cii.involvement, "DRIVER") AS involvement,
vi."Vehicle Direction" AS vehicle_direction

from pre_feature_eng_crs_ii cii
left outer join vehicle_information vi 
on  cii.acc_uuid = vi.acc_uuid  --13751, inner(10129)
and cii.name = vi."Driver Name" 
and cii.age = vi."Driver Age" 
and cii."city_state" = vi."Driver City/State"
'''

In [47]:
c = conn.cursor()
c.execute(sql)

In [48]:
df = dbc.read_sql(mshp_db_conn, f"SELECT * FROM pre_feature_eng") 

In [29]:
from ydata_profiling import ProfileReport

Pandas backend loaded 1.5.3
Numpy backend loaded 1.23.5
Pyspark backend NOT loaded
Python backend loaded


In [30]:
table_name = "pre_feature_eng"
profile = ProfileReport(df, title=f"Profiling Report for {table_name}")

In [31]:
profile.to_file(f"Profiling Report for {table_name}")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

c:\Users\lgarzia\Documents\GitHub\explainability\venv\lib\site-packages\ydata_profiling\profile_report.py:355: UserWarning: Extension  not supported. For now we assume .html was intended. To remove this warning, please use .html or .json.
  warnings.warn(


Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [49]:
df.count() # 13724

acc_rpt_num          13749
acc_uuid             13749
name                 13749
injury_type          13749
age                  13749
city_state           13719
safety_device        13749
date                 13749
time                 13749
troop                13749
gender               13719
veh_num              13719
involvement          13749
vehicle_direction    10780
dtype: int64

In [33]:
# Question - still missing gender - what of 55
    # ANS - fixed issue with join on Injury Type - no have dupes that need to be cleaned

# Question - 3 missing vehicle direction - can we obtain the remainder with a join on acc_uuid & veh_num
# Question - would removing Juveniles help
# ANS - yes -> proceed with removing for analysis Name != 'JUVENILE,'

In [36]:
import numpy as np
df[df.gender.isnull()].head(n=10)
# '831f4197-f4aa-4cfe-b45b-b5fb865a04f0' <-- involvement isn't right
# appears Injury Type in None versus NO INJURY 

,acc_rpt_num,acc_uuid,name,injury_type,age,city_state,safety_device,date,time,troop,gender,veh_num,involvement,vehicle_direction
143,220121269,831f4197-f4aa-4cfe-b45b-b5fb865a04f0,"MILBOURN, KAYTIE L",NO INJURY,20,"KIDDER, MO",UNKNOWN,03/10/2022,5:33PM,H,None,<NA>,DRIVER,None
144,220121269,3506e870-f374-4aa9-93fb-cd0e42dbabb0,"ORR, BRIDGETTE L",NO INJURY,22,"HAMILTON, MO",NO,03/10/2022,5:33PM,H,None,<NA>,DRIVER,None
628,220147421,e3b1166a-9226-4490-a23d-6dbafd4d8eb9,"UNKNOWN, UNKNOWN",NO INJURY,UNK,None,UNKNOWN,03/25/2022,8:05AM,C,None,<NA>,DRIVER,None
629,220147363,58235742-d6c7-414e-b151-c853f90922ca,"HESTER, STEPHEN L",NO INJURY,18,"MOUNTAIN VIEW, MO",YES,03/25/2022,7:30AM,G,None,<NA>,DRIVER,None
687,220154476,278223ac-acf2-49ef-a055-05056f55bb47,"UNKNOWN, UNKNOWN",NO INJURY,UNK,None,UNKNOWN,03/28/2022,3:00PM,D,None,<NA>,DRIVER,None
891,220164885,13725b6c-5692-4a0e-90b5-e841013fd5ed,"UNK, UNK",NO INJURY,UNK,None,UNKNOWN,04/03/2022,12:37PM,C,None,<NA>,DRIVER,None
906,220164473,8bee3e66-7112-40e7-a6ef-6a9958b732e7,"UNKNOWN, UNKNOWN",NO INJURY,UNK,None,UNKNOWN,04/03/2022,2:14AM,D,None,<NA>,DRIVER,None
1063,220178007,adaed995-a897-4af2-af55-b5e567845edb,"UNKNOWN, UNKNOWN",NO INJURY,UNK,None,UNKNOWN,04/09/2022,8:30PM,G,None,<NA>,DRIVER,None
1426,220199768,de62c379-2cec-42fe-a6ce-a826bbbc48ed,"JUVENILE,",NO INJURY,0,"LATHROP, MO",YES,04/21/2022,3:20PM,A,None,<NA>,DRIVER,None
1639,220209109,f25715fb-4098-4930-a7d6-4d49609c7660,"PAYNE, EVELYN N",NO INJURY,39,"MARSHALL, MO",YES,04/26/2022,11:59AM,A,None,<NA>,DRIVER,None
